<a href="https://colab.research.google.com/github/nehanawar025/EDA-BLACK-FRIDAY-SALES/blob/main/Colab/EDA_Black_Friday_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

Importing data from Github

In [ ]:
!wget -O data.csv https://raw.githubusercontent.com/nehanawar025/EDA-BLACK-FRIDAY-SALES/main/Dataset/walmart_data.csv

--2024-09-08 05:07:35--  https://raw.githubusercontent.com/nehanawar025/EDA-BLACK-FRIDAY-SALES/main/Dataset/walmart_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23578063 (22M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  22.49M  --.-KB/s    in 0.1s    

2024-09-08 05:07:36 (161 MB/s) - ‘data.csv’ saved [23578063/23578063]



In [ ]:
df = pl.read_csv('data.csv')

In [ ]:
df = df.select(
    pl.col("User_ID").cast(pl.Utf8),
    pl.col("Product_ID"),
    pl.col("Gender"),
    pl.col("Age").alias("Age_groups"),
    pl.col("Occupation").cast(pl.Utf8),
    pl.col("City_Category"),
    pl.col("Stay_In_Current_City_Years"),
    pl.col("Marital_Status").cast(pl.Utf8),
    pl.col("Product_Category").cast(pl.Utf8),
    pl.col("Purchase")
)

In [ ]:
df.head(10)

User_ID,Product_ID,Gender,Age_groups,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category,Purchase
str,str,str,str,str,str,str,str,str,i64
"""1000001""","""P00069042""","""F""","""0-17""","""10""","""A""","""2""","""0""","""3""",8370
"""1000001""","""P00248942""","""F""","""0-17""","""10""","""A""","""2""","""0""","""1""",15200
"""1000001""","""P00087842""","""F""","""0-17""","""10""","""A""","""2""","""0""","""12""",1422
"""1000001""","""P00085442""","""F""","""0-17""","""10""","""A""","""2""","""0""","""12""",1057
"""1000002""","""P00285442""","""M""","""55+""","""16""","""C""","""4+""","""0""","""8""",7969
"""1000003""","""P00193542""","""M""","""26-35""","""15""","""A""","""3""","""0""","""1""",15227
"""1000004""","""P00184942""","""M""","""46-50""","""7""","""B""","""2""","""1""","""1""",19215
"""1000004""","""P00346142""","""M""","""46-50""","""7""","""B""","""2""","""1""","""1""",15854
"""1000004""","""P0097242""","""M""","""46-50""","""7""","""B""","""2""","""1""","""1""",15686


---

**Gender-AgeGroup-wise Distribution**

In [22]:
f_age_wise = (
    df.filter(pl.col('Gender')=='F')
    .groupby(['Age_groups'])
    .agg(
        pl.col('User_ID').unique().count().alias('Total_Female_customers')
    )
    .sort(['Age_groups'], descending=False)
)
print(f_age_wise)

shape: (7, 2)
┌────────────┬────────────────────────┐
│ Age_groups ┆ Total_Female_customers │
│ ---        ┆ ---                    │
│ str        ┆ u32                    │
╞════════════╪════════════════════════╡
│ 0-17       ┆ 78                     │
│ 18-25      ┆ 287                    │
│ 26-35      ┆ 545                    │
│ 36-45      ┆ 333                    │
│ 46-50      ┆ 182                    │
│ 51-55      ┆ 142                    │
│ 55+        ┆ 99                     │
└────────────┴────────────────────────┘


<ipython-input-22-832fa59536db>:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(['Age_groups'])


In [27]:
f_age_wise_pandas = f_age_wise.to_pandas()

fig = px.bar(
    f_age_wise_pandas,
    x = "Age_groups",
    y = "Total_Female_customers",
    title = "Age group wise Female Customers",
    color_discrete_sequence=["#da627d"]
)

fig.update_traces(
    hovertemplate = '<b>Age_group: %{x}</b><br><b>Total_Female_customers: %{y}</b>'
)

fig.update_layout(
    height = 500,
    width = 650,
    bargap = 0.25,
    xaxis_title = None,
    yaxis_title = None
)

fig.show()

In [21]:
m_age_wise = (
    df.filter(pl.col('Gender')=='M')
    .groupby(['Age_groups'])
    .agg(
        pl.col('User_ID').unique().count().alias('Total_Male_customers')
    )
    .sort(['Age_groups'], descending=False)
)
print(m_age_wise)

shape: (7, 2)
┌────────────┬──────────────────────┐
│ Age_groups ┆ Total_Male_customers │
│ ---        ┆ ---                  │
│ str        ┆ u32                  │
╞════════════╪══════════════════════╡
│ 0-17       ┆ 140                  │
│ 18-25      ┆ 782                  │
│ 26-35      ┆ 1508                 │
│ 36-45      ┆ 834                  │
│ 46-50      ┆ 349                  │
│ 51-55      ┆ 339                  │
│ 55+        ┆ 273                  │
└────────────┴──────────────────────┘


<ipython-input-21-32ac49298e21>:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(['Age_groups'])


---

**Which age group contributes the most to the total sales?**

In [23]:
result1 = (
    df.group_by(["Age_groups"])
    .agg(Total_sales_per_age_groups = pl.col("Purchase").sum())
    .sort(["Total_sales_per_age_groups"], descending=True)
)

result1

Age_groups,Total_sales_per_age_groups
str,i64
"""26-35""",2031770578
"""36-45""",1026569884
"""18-25""",913848675
"""46-50""",420843403
"""51-55""",367099644
"""55+""",200767375
"""0-17""",134913183


In [ ]:
result1_pandas = result1.to_pandas()

fig = px.bar(
    result1_pandas,
    x = "Age_groups",
    y = "Total_sales_per_age_groups",
    title = "Total Sales by Age Groups",
    color_discrete_sequence=["#565264"]
)

fig.update_traces(
    hovertemplate = '<b>Age Group: %{x}</b><br><b>Total Sales: %{y}</b>'
)

fig.update_layout(
    width = 700,
    bargap = 0.25,
    yaxis_title = None
)

fig.show()

We can see "26-35" age group is the top age group with 2.032B customers.

---

**Top 3 products purchased by top customer age group**

In [ ]:
result2 = (
    df.filter(pl.col("Age_groups") == '26-35')
    .group_by(["Product_Category"])
    .agg(
        Total_sales = pl.col("Purchase").sum()
    )
    .sort(["Total_sales"], descending=True)
    .top_k(3, by = 'Total_sales')
)

result2

Product_Category,Total_sales
str,i64
"""1""",783813459
"""5""",379702493
"""8""",327523339


In [ ]:
result2_pandas = result2.to_pandas()

fig = px.bar(
    result2_pandas,
    x = "Product_Category",
    y = "Total_sales",
    title = "Top 3 Product Categories for Age Group 26-35",
    color_discrete_sequence=["#565264"]
)

fig.update_traces(
    hovertemplate = '<b>Product Category: %{x}</b><br><b>Total Sales: %{y}</b>'
)

fig.update_layout(
    height = 500,
    width = 500,
    bargap = 0.25,
    yaxis_title = None
)

fig.show()

1, 5, 8 are the top 3 product categories. Among top 3 categories, 1 is best-selling.

---

**What is the most popular product category among male customers in the 26-35 age group during Black Friday, and how does it compare to female customers in the same age group?**

In [ ]:
result3_m = (
    df.filter(
        (pl.col("Age_groups") == '26-35') &
        (pl.col("Gender") == 'M')
    )
    .group_by(["Product_Category"])
    .agg(
        Total_sales = pl.col("Purchase").sum()
    )
    .sort(["Product_Category"], descending=[True])
    .top_k(3, by = "Total_sales")
    )

result3_m

Product_Category,Total_sales
str,i64
"""1""",656168234
"""5""",275656900
"""8""",232994930


In [ ]:
result3_m_pandas = result3_m.to_pandas()

fig = px.bar(
    result3_m_pandas,
    x = "Product_Category",
    y = "Total_sales",
    title = "Top 3 Product Categories for Male Customers",
    color_discrete_sequence=["#2986cc"]
)

fig.update_traces(
    hovertemplate = '<b>Product Category: %{x}</b><br><b>Total Sales: %{y}</b>'
)

fig.update_layout(
    height = 500,
    width = 500,
    bargap = 0.25,
    yaxis_title = None
)

fig.show()

In [ ]:
result3_f = (
    df.filter(
        (pl.col("Age_groups") == '26-35') &
        (pl.col("Gender") == 'F')
    )
    .group_by(["Product_Category"])
    .agg(
        Total_sales = pl.col("Purchase").sum()
    )
    .sort(["Product_Category"], descending=[True])
    .top_k(3, by = "Total_sales")
    )

result3_f

Product_Category,Total_sales
str,i64
"""1""",127645225
"""5""",104045593
"""8""",94528409


In [ ]:
result3_f_pandas = result3_f.to_pandas()

fig = px.bar(
    result3_f_pandas,
    x = "Product_Category",
    y = "Total_sales",
    title = "Top 3 Product Categories for Female Customers",
    color_discrete_sequence=["#da627d"]
)

fig.update_traces(
    hovertemplate = '<b>Product Category: %{x}</b><br><b>Total Sales: %{y}</b>'
)

fig.update_layout(
    height = 500,
    width = 500,
    bargap = 0.25,
    yaxis_title = None
)

fig.show()

In [ ]:
top_product_categories = result2["Product_Category"].to_list()

# Filter `df` using the extracted categories
result3 = (
    df.filter(
        (pl.col("Age_groups") == '26-35') &
        (pl.col("Product_Category").is_in(top_product_categories))
    )
    .group_by(["Product_Category", "Gender"])
    .agg(
        Total_sales = pl.col("Purchase").sum()
    )
    .sort(["Product_Category","Gender"], descending=[False,True])
)

result3

Product_Category,Gender,Total_sales
str,str,i64
"""1""","""M""",656168234
"""1""","""F""",127645225
"""5""","""M""",275656900
"""5""","""F""",104045593
"""8""","""M""",232994930
"""8""","""F""",94528409


In [ ]:
result3_df = result3.to_pandas()

fig = px.bar(result3_df,
             x="Product_Category",
             y="Total_sales",
             color="Gender",
             color_discrete_map={"M": "#2986cc", "F": "#da627d"},
             barmode="group",
             title="Top Product Categories by Gender for 26-35 Age Group")

fig.update_traces(
    hovertemplate = '<b>Product Category: %{x}</b><br><b>Total Sales: %{y}</b>'
)

fig.update_layout(
    height = 500,
    width = 700,
    bargap = 0.25,
    yaxis_title = None
)

fig.show()